In [1]:
from Modules import DynamicSymbol, StaticSymbol, DynamicSymbols, StaticSymbols, diff_t, Drehmatrix, DynamicSystem, StaticSystem, disp
import symengine as se
import sympy

import re

se.init_printing()
sympy.init_printing(use_latex='mathjax')

In [2]:
iG = se.Symbol("i_G" , real = True)
c = se.Symbol("c" , real = True)
s = se.Symbol("s" , real = True)
m = se.Symbol("m" , real = True)
CA = se.Symbol("C_A" , real = True)
CM = se.Symbol("C_M" , real = True)
km = se.Symbol("k_m" , real = True)
[iG,c,s,m,CA,CM,km] = StaticSymbols(["i_G","c","s","m","C_A","C_M","k_m"])
params = [iG,c,s,m,CA,CM,km]

In [3]:
# q_var = DynamicSymbols(["q_M","q_A"],1,2,True)
q_var = DynamicSymbol("q",2,2)
u_var = DynamicSymbol("M_{Mot}",1,0)

vars = DynamicSymbols(["q_M","q_A"],1,2)
[q,q_dot,q_ddot] = vars
[q,q_dot,q_ddot] = [se.Matrix(q),se.Matrix(q_dot),se.Matrix(q_ddot)]
[[qM,qA],[qM_dot, qA_dot],[qM_ddot, qA_ddot]] = vars


u = u_var.vars
#[q,q_dot,q_ddot] = q_var.vars
#[[qM,qA],[qM_dot, qA_dot],[qM_ddot, qA_ddot]] = q_var.vars
#display(q_var, u_var)
M_Mot = u[0]

In [4]:
K_J_AS = se.diag(0,0,CA)
K_J_AS

[0, 0, 0]
[0, 0, 0]
[0, 0, C_{{A}_{1}}^{}]

In [5]:
K_J_MS = se.zeros(3)
K_J_MS[2,2] = CM


K_J_MS

[0, 0, 0]
[0, 0, 0]
[0, 0, C_{{M}_{1}}^{}]

In [6]:
R_IK = Drehmatrix(se.Matrix([0,0,qM]))
display(R_IK)

[cos(q_{M}^{}(t)), -sin(q_{M}^{}(t)), 0]
[sin(q_{M}^{}(t)), cos(q_{M}^{}(t)), 0]
[0, 0, 1]

In [7]:
K_r_s = se.Matrix([s,0,0])
K_r_s

[s_{1}^{}]
[0]
[0]

In [8]:
K_w_K = se.Matrix([0,0,qA_dot+qM_dot])
K_w_K

[0]
[0]
[\dot{q}_{A}^{}(t) + \dot{q}_{M}^{}(t)]

In [9]:
K_w_M = se.Matrix([0,0,iG*qM_dot])
K_w_M

[0]
[0]
[\dot{q}_{M}^{}(t)*i_{{G}_{1}}^{}]

In [10]:
K_v_S = diff_t(K_r_s) + K_w_K.cross(K_r_s)
K_v_S

[0]
[(\dot{q}_{A}^{}(t) + \dot{q}_{M}^{}(t))*s_{1}^{}]
[0]

In [11]:
K_p = m * K_v_S
K_p

[0]
[(\dot{q}_{A}^{}(t) + \dot{q}_{M}^{}(t))*m_{1}^{}*s_{1}^{}]
[0]

In [12]:
K_L_A = K_J_AS * K_w_K
K_L_A

[0]
[0]
[(\dot{q}_{A}^{}(t) + \dot{q}_{M}^{}(t))*C_{{A}_{1}}^{}]

In [13]:
K_L_M = K_J_MS * K_w_M
K_L_M

[0]
[0]
[C_{{M}_{1}}^{}*\dot{q}_{M}^{}(t)*i_{{G}_{1}}^{}]

In [14]:
K_Me_A = se.Matrix([0,0,-qA * c])
K_Me_A

[0]
[0]
[-c_{1}^{}*q_{A}^{}(t)]

In [15]:
K_Me_M = se.Matrix([0,0,M_Mot]) - K_Me_A / iG
K_Me_M

[0]
[0]
[c_{1}^{}*q_{A}^{}(t)/i_{{G}_{1}}^{} + M_{Mot}^{}(t)]

In [16]:
display(diff_t(K_p), K_p)
part1 = K_v_S.jacobian(q_dot).transpose() * (diff_t(K_p) + K_w_K.cross(K_p))

disp(part1)

[0]
[(\ddot{q}_{A}^{}(t) + \ddot{q}_{M}^{}(t))*m_{1}^{}*s_{1}^{}]
[0]

[0]
[(\dot{q}_{A}^{}(t) + \dot{q}_{M}^{}(t))*m_{1}^{}*s_{1}^{}]
[0]

⎡                   2                                            ⎤
⎢m_{1}__{}⋅s_{1}__{} ⋅(\ddot{q}_{A}__{}(t) + \ddot{q}_{M}__{}(t))⎥
⎢                                                                ⎥
⎢                   2                                            ⎥
⎣m_{1}__{}⋅s_{1}__{} ⋅(\ddot{q}_{A}__{}(t) + \ddot{q}_{M}__{}(t))⎦

In [17]:
part2 = K_w_K.jacobian(q_dot).transpose() * (diff_t(K_L_A) + K_w_K.cross(K_L_A) - K_Me_A)
part2

[c_{1}^{}*q_{A}^{}(t) + (\ddot{q}_{A}^{}(t) + \ddot{q}_{M}^{}(t))*C_{{A}_{1}}^{}]
[c_{1}^{}*q_{A}^{}(t) + (\ddot{q}_{A}^{}(t) + \ddot{q}_{M}^{}(t))*C_{{A}_{1}}^{}]

In [18]:
part3 = K_w_M.jacobian(q_dot).transpose() * (diff_t(K_L_M) + K_w_M.cross(K_L_M) - K_Me_M)
part3

[(\ddot{q}_{M}^{}(t)*C_{{M}_{1}}^{}*i_{{G}_{1}}^{} - (c_{1}^{}*q_{A}^{}(t)/i_{{G}_{1}}^{} + M_{Mot}^{}(t)))*i_{{G}_{1}}^{}]
[0]

In [19]:
parts = part1 + part2 + part3
disp(parts)

⎡                                                                             
⎢C_{{A}_{1}}__{}⋅(\ddot{q}_{A}__{}(t) + \ddot{q}_{M}__{}(t)) + c_{1}__{}⋅q_{A}
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎣                                                         C_{{A}_{1}}__{}⋅(\dd

                          ⎛                                                   
__{}(t) + i_{{G}_{1}}__{}⋅⎜C_{{M}_{1}}__{}⋅i_{{G}_{1}}__{}⋅\ddot{q}_{M}__{}(t)
                          ⎝                                                   
                                                                              
                                                                              
ot{q}_{A}__{}(t) + \ddot{q}_{M}__{}(t)) + c_{1}__{}⋅q_{A}__{}(t) + m_{1}__{}⋅s

   c_{1}__{}⋅q_{A}__{}(t)                 ⎞       

In [20]:
M = parts.jacobian(q_ddot)
disp(M)

⎡                                                 2                      2    
⎢C_{{A}_{1}}__{} + C_{{M}_{1}}__{}⋅i_{{G}_{1}}__{}  + m_{1}__{}⋅s_{1}__{}   C_
⎢                                                                             
⎢                                                      2                      
⎣                 C_{{A}_{1}}__{} + m_{1}__{}⋅s_{1}__{}                     C_

                                   2⎤
{{A}_{1}}__{} + m_{1}__{}⋅s_{1}__{} ⎥
                                    ⎥
                                   2⎥
{{A}_{1}}__{} + m_{1}__{}⋅s_{1}__{} ⎦

In [21]:
g = parts - M @  q_ddot
g = g.expand()
g

[-M_{Mot}^{}(t)*i_{{G}_{1}}^{}]
[c_{1}^{}*q_{A}^{}(t)]

In [22]:
f = q_dot.col_join((-M.inv() * g.expand()))

In [23]:
x_vars = q.col_join(q_dot)
h = se.Matrix([[qM + qA],[(qM_dot + qA_dot * qA)]])

dynSys = DynamicSystem(x_vars,u)
dynSys.addStateEquations(f)
dynSys.addOutput(h,"y")
dynSys.addOutput(h,"l")
dynSys.addInput(u,"u")
dynSys.addParameter(params)
#sys = Robotik_System(f,h,q_var,u_var,params,False)

In [24]:
[A,B,C,D] = dynSys.linearize(se.Matrix([0,0,0,0]),se.Matrix([0]))
disp(A,B,C,D)


⎡0                                                                            
⎢                                                                             
⎢0                                                                            
⎢                                                                             
⎢                                                                             
⎢                                                                          c_{
⎢0  ──────────────────────────────────────────────────────────────────────────
⎢                                                                             
⎢                                                                             
⎢   ⎛                                                 2                      2
⎢   ⎝C_{{A}_{1}}__{} + C_{{M}_{1}}__{}⋅i_{{G}_{1}}__{}  + m_{1}__{}⋅s_{1}__{} 
⎢                                                                             
⎢                                                   

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                ⎛                                                            
⎢                ⎜                                                            
⎢                ⎜                                                            
⎢i_{{G}_{1}}__{}⋅⎜────────────────────────────────────────────────────────────
⎢                ⎜                                                            
⎢                ⎜                                                            
⎢                ⎜⎛                                                 2         
⎢                ⎜⎝C_{{A}_{1}}__{} + C_{{M}_{1}}__{}⋅i_{{G}_{1}}__{}  + m_{1}_
⎢                ⎜                                  

⎡1  1  0  0⎤
⎢          ⎥
⎢0  0  1  0⎥
⎢          ⎥
⎢1  1  0  0⎥
⎢          ⎥
⎣0  0  1  0⎦

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [25]:
dynSys.write_ABCD_to_File("ss_matrices")
dynSys.write_init_File("init_params")
dynSys.write_SFunction("sfun_test")
dynSys.write_MFunctions("Test")